In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import sys
from tqdm import tqdm
sys.path.insert(0, '/mnt/lareaulab/cfbuenabadn/psix_project/psix/psix/')
import psix
from matplotlib.gridspec import GridSpec

from scipy.special import logit, expit
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import auc

cm = 1/2.54

In [4]:
from scipy.stats import kruskal
from statsmodels.stats.multitest import multipletests


def run_kw(exon_psi, labels):
    kw_input = []
    for l in labels:
        kw_input.append(list(exon_psi[l]))
        
    if len(labels) == 2:
        return kruskal(kw_input[0], kw_input[1], nan_policy='omit')
    if len(labels) == 3:
        return kruskal(kw_input[0], kw_input[1], kw_input[2], nan_policy='omit')
    if len(labels) == 4:
        return kruskal(kw_input[0], kw_input[1], kw_input[2], kw_input[3], nan_policy='omit')
    if len(labels) == 5:
        return kruskal(kw_input[0], kw_input[1], kw_input[2], kw_input[3], kw_input[4], nan_policy='omit')
    if len(labels) == 6:
        return kruskal(kw_input[0], kw_input[1], kw_input[2], kw_input[3], kw_input[4], kw_input[5], nan_policy='omit')
    if len(labels) == 7:
        return kruskal(kw_input[0], kw_input[1], kw_input[2], kw_input[3], kw_input[4], kw_input[5], kw_input[6], nan_policy='omit')
    if len(labels) == 8:
        return kruskal(kw_input[0], kw_input[1], kw_input[2], kw_input[3], kw_input[4], kw_input[5], kw_input[6], kw_input[7], nan_policy='omit')
    
def run_kw_dset(psi_table, labels, exon_list):
    kw_output = pd.DataFrame()
    kw_score = []
    pvals = []
    for exon in tqdm(exon_list, position=0, leave=True):
        score, p = run_kw(psi_table.loc[exon], labels)
        kw_score.append(score)
        pvals.append(p)
    kw_output['KW_score'] = kw_score
    kw_output['pvals'] = pvals
    kw_output['qvals'] = multipletests(pvals, method='fdr_bh')[1]
    kw_output.index = exon_list
    return kw_output



In [6]:
psix_three_lineage_1 = psix.Psix()
psix_three_lineage_1.process_rnaseq(
        'three_lineages/processed_tables/SE_counts_0.1.tab.gz',
        'three_lineages/processed_tables/constitutive_introns_0.1.tab.gz',
        'three_lineages/processed_tables/tpm_0.1.tab.gz',
        minJR = 1,
        minCell=1,
        min_observed = 0.25)

psix_three_lineage_1.compute_psix_scores(latent='three_lineages/processed_tables/pc2_rd.tab.gz', n_jobs=25, 
                                n_random_exons=2000, n_neighbors=100)

psix_three_lineage_05 = psix.Psix()
psix_three_lineage_05.process_rnaseq(
        'three_lineages/processed_tables/SE_counts_0.05.tab.gz',
        'three_lineages/processed_tables/constitutive_introns_0.05.tab.gz',
        'three_lineages/processed_tables/tpm_0.05.tab.gz',
        minJR = 1,
        minCell=1,
        min_observed = 0.25)

psix_three_lineage_05.compute_psix_scores(latent='three_lineages/processed_tables/pc2_rd.tab.gz', n_jobs=25, 
                                n_random_exons=2000, n_neighbors=100)

psix_three_lineage_01 = psix.Psix()
psix_three_lineage_01.process_rnaseq(
        'three_lineages/processed_tables/SE_counts_0.01.tab.gz',
        'three_lineages/processed_tables/constitutive_introns_0.01.tab.gz',
        'three_lineages/processed_tables/tpm_0.01.tab.gz',
        minJR = 1,
        minCell=1,
        min_observed = 0.25)

psix_three_lineage_01.compute_psix_scores(latent='three_lineages/processed_tables/pc2_rd.tab.gz', n_jobs=25, 
                                n_random_exons=2000, n_neighbors=100)


meta = pd.read_csv('three_lineages/sim_output/meta.tab.gz', sep='\t')
meta.index=['cell_'+str(i+1) for i in range(1000)]

pop_1 = meta.loc[meta['pop'] == '4_1'].index
pop_2 = meta.loc[meta['pop'] == '4_5'].index
pop_3 = meta.loc[meta['pop'] == '5_2'].index
pop_4 = meta.loc[meta['pop'] == '5_3'].index
labels = [pop_1, pop_2, pop_3, pop_4]

three_kw_1 = run_kw_dset(psix_three_lineage_1.adata.uns['psi'][psix_three_lineage_1.psix_results.index].T, 
                         labels, psix_three_lineage_1.psix_results.index)

three_kw_05 = run_kw_dset(psix_three_lineage_05.adata.uns['psi'][psix_three_lineage_05.psix_results.index].T, 
                         labels, psix_three_lineage_05.psix_results.index)

three_kw_01 = run_kw_dset(psix_three_lineage_01.adata.uns['psi'][psix_three_lineage_01.psix_results.index].T, 
                         labels, psix_three_lineage_01.psix_results.index)

l1 = []
with open('three_lineages/sim_output/l1_diff.txt', 'r') as l1_file:
    for i in l1_file:
        l1.append(int(i.rstrip()))
        
l2 = []
with open('three_lineages/sim_output/l2_diff.txt', 'r') as l2_file:
    for i in l2_file:
        l2.append(int(i.rstrip()))
        
l3 = []
with open('three_lineages/sim_output/l3_diff.txt', 'r') as l3_file:
    for i in l3_file:
        l3.append(int(i.rstrip()))
        
l1 = np.array(l1)
l2 = np.array(l2)
l3 = np.array(l3)

ds = ((l1 + l2 + l3)>0).astype(int)

gearyc_3l_1 = pd.read_csv('three_lineages/gearyc_0.1.tab.gz', sep='\t', index_col=0)
gearyc_3l_05 = pd.read_csv('three_lineages/gearyc_0.05.tab.gz', sep='\t', index_col=0)
gearyc_3l_01 = pd.read_csv('three_lineages/gearyc_0.01.tab.gz', sep='\t', index_col=0)

100%|██████████| 1931/1931 [00:14<00:00, 137.28it/s]
